In [73]:
# import h5 reader
import h5py
import numpy as np
work_path = "/home/shengduo/pylith-developer/build/debug/pylith-nonRegSlipLawWithVaryingB/examples/bar_shearwave/quad4/"
h5_path = work_path + "output/fault/"

In [76]:
# Class RunABatch
class RunABatch:
    # Constructor
    def __init__(self, input_set, work_path, FourierTerms = 16):
        # Initialize data paths and batch parameters
        self.work_path = work_path
        self.input_set = input_set
        self.h5_path = work_path + "output/fault/"
        self.FourierTerms = FourierTerms
        
        # Flag for whether cases have been run
        self.casesExcuted = False
        
        # Run cases
        self.runCases(self.input_set)
        
        # Get obsevations
        self.Observations = getObservations(self.input_set)
        
    # Function runCases
    def runCases(self, input_set):
        shell_path = self.work_path + "RunJobsJP.sh"
        shellRead = open(shell_path, 'r')
        list_of_lines = shellRead.readlines()
        shellRead.close()

        AA = [ele[0] for ele in input_set]
        BB = [ele[1] for ele in input_set]

        list_of_lines[9] = "AA=" + str(tuple(AA)).replace(',', '') + "\n"
        list_of_lines[10] = "BB=" + str(tuple(BB)).replace(',', '') + "\n"

        shellWrite = open(shell_path, 'w')
        shellWrite.writelines(list_of_lines)
        shellWrite.close()

        # Run the cases
        !source $shell_path
        
        self.casesExcuted = True
        
        # Return from shell
        return
    
    
    # Function getObservations for the input_set
    def getObservations(self, input_set):
        # Initialize Observations
        Observations = []
        
        # Check if the cases have been excuted
        if self.casesExcuted == False:
            return Observations
        
        # Loop through all Inputs
        for input_ele in input_set:
            # Open the file
            h5_file = self.h5_path + "A" + str(input_ele[0]) + "_B" + str(input_ele[1]) + "-fault.h5"
            f = h5py.File(h5_file, 'r')

            # Get time
            time = np.array(f['time']).reshape([-1])
            time = time - np.min(time)
            nOfTSteps = time.shape[0]

            # Get Slip rates
            SlipRates = np.array(f['vertex_fields']['slip_rate'])
            Vx = SlipRates[:, :, 0].transpose()
            Vy = SlipRates[:, :, 1].transpose()
            nOfNodes = Vx.shape[0]

            # Find the Fourier coefficients
            FourierTerms = self.FourierTerms
            T = np.max(time)

            # Compute the Fourier terms
            Ks = np.array(range(FourierTerms))
            coskPiTt = np.cos(Ks.reshape([-1, 1]) * np.pi / T * time)
            VxcoskPiTt = np.concatenate([coskPiTt * Vxi.reshape([1, -1]) for Vxi in Vx], 0)

            # Compute the fourier coefficients
            # print('time.shape: ', time.shape)
            observation = np.trapz(VxcoskPiTt, x=time)

            # Append the result from this file
            Observations.append(observation)
            
        return Observations
    
    

In [77]:
# Input example of getting observations
AA = np.linspace(0.002, 0.012, 6)
BB = AA + 0.004

# Get input set
input_set = [[AA[i], BB[i]] for i in range(len(AA))]

# Test class RunABatch
myBatch = RunABatch(input_set, work_path)

FileNotFoundError: [Errno 2] No such file or directory: '/home/shengduo/pylith-developer/build/debug/pylith-nonRegSlipLawWithVaryingB/examples/bar_shearwave/quad4/RunJobsJP.sh'

In [17]:
# Function getObservations:
def getObservations(h5_path, input_set, FourierTerms):
    Observations = []
    # Loop through all Inputs
    for input_ele in input_set:
        # Open the file
        h5_file = h5_path + "A" + str(input_ele[0]) + "_B" + str(input_ele[1]) + "-fault.h5"
        f = h5py.File(h5_file, 'r')
        
        # Get time
        time = np.array(f['time']).reshape([-1])
        time = time - np.min(time)
        nOfTSteps = time.shape[0]

        # Get Slip rates
        SlipRates = np.array(f['vertex_fields']['slip_rate'])
        Vx = SlipRates[:, :, 0].transpose()
        Vy = SlipRates[:, :, 1].transpose()
        nOfNodes = Vx.shape[0]

        # Find the Fourier coefficients
        FourierTerms = 16
        T = np.max(time)

        # Compute the Fourier terms
        Ks = np.array(range(FourierTerms))
        coskPiTt = np.cos(Ks.reshape([-1, 1]) * np.pi / T * time)
        VxcoskPiTt = np.concatenate([coskPiTt * Vxi.reshape([1, -1]) for Vxi in Vx], 0)

        # Compute the fourier coefficients
        # print('time.shape: ', time.shape)
        observation = np.trapz(VxcoskPiTt, x=time)
        
        # Append the result from this file
        Observations.append(observation)
    return Observations

In [32]:
# Function runCases
def runCases(work_path, input_set):
    shell_path = work_path + "RunJobsJP.sh"
    shellRead = open(shell_path, 'r')
    list_of_lines = shellRead.readlines()
    shellRead.close()
    
    AA = [ele[0] for ele in input_set]
    BB = [ele[1] for ele in input_set]
    
    list_of_lines[9] = "AA=" + str(tuple(AA)).replace(',', '') + "\n"
    list_of_lines[10] = "BB=" + str(tuple(BB)).replace(',', '') + "\n"

    shellWrite = open(shell_path, 'w')
    shellWrite.writelines(list_of_lines)
    shellWrite.close()
    
    # Run the cases
    print('Shell path: ', shell_path)
    !source $shell_path
    
    return

In [68]:
# Example of getting observations
AA = np.linspace(0.002, 0.012, 6)
BB = AA + 0.004

# Get input set
input_set = [[AA[i], BB[i]] for i in range(len(AA))]

# Run the cases
runCases(work_path, input_set)

# Get observations
res = getObservations(h5_path, input_set, 16)

NameError: name 'runCases' is not defined

In [34]:
res

[array([-276.72222533,    9.21417284,    9.28709317,    9.03502741,
           9.01451615,    9.00146222,    8.60317807,    8.54807941,
           8.53598392,    7.83240749,    8.08698523,    7.65785586,
           7.0987034 ,    7.40963169,    6.62068071,    6.37692284,
        -276.2554538 ,    9.69130499,    9.73530519,    9.47562458,
           9.40846043,    9.37064061,    8.91104473,    8.81697552,
           8.72762581,    7.98022883,    8.14300881,    7.66002745,
           7.01578593,    7.25178855,    6.38254256,    6.07746384,
        -276.72222533,    9.21417284,    9.28709317,    9.03502741,
           9.01451615,    9.00146222,    8.60317807,    8.54807941,
           8.53598392,    7.83240749,    8.08698523,    7.65785586,
           7.0987034 ,    7.40963169,    6.62068071,    6.37692284]),
 array([-274.96340531,   10.8731168 ,   11.02632182,   10.64961171,
          10.70580393,   10.53617546,   10.17650899,    9.99433927,
           9.95847399,    9.07379865,    9.400

In [72]:
# Import Gaussian-regression related functions
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import RBF

# Pre-process the data
from sklearn import preprocessing

# function train_GP
class GP_predictor:
    # Constructor
    def __init__(self, 
                 input_set, 
                 observation_set, 
                 GPkernel = 1 * RBF(length_scale=1.0, length_scale_bounds=(1e-1, 1e2)), 
                 n_optimizers = 9):
        # Scale input data
        self.input_scaler = preprocessing.StandardScaler()
        self.input_scaler.fit(np.array(input_set))
        
        # Scale output data
        self.observation_scaler = preprocessing.StandardScaler()
        self.observation_scaler.fit(np.array(observation_set))
        
        # Fit Gaussian process
        self.GP = GaussianProcessRegressor(kernel = GPkernel, n_restarts_optimizer = n_optimizers)
        self.GP.fit(self.input_scaler.transform(np.array(input_set)), 
                    self.observation_scaler.transform(np.array(observation_set)))
        
    # Predict on a new input set
    def predict(self, new_input_set):
        # Predict new observation
        new_observation = self.observation_scaler.inverse_transform(
            self.GP.predict(
                self.input_scaler.transform(np.array(new_input_set))
            )
        )
        
        return new_observation
        
    

In [ ]:
# Generate a set, run cases, get observations

In [ ]:
myGP = GP_predictor(input_set, )